## Project Camera Following


In [1]:
import numpy as np
import cv2


cap = cv2.VideoCapture(0)
while True:
    success, img = cap.read()
    if success:
        # Get height and width of webcam frame
        height, width = img.shape[:2]
        # Define ROI Box Dimensions
        top_left_x = width / 3
        top_left_y = (height / 2) + (height / 4)
        bottom_right_x = (width / 3) * 2
        bottom_right_y = (height / 2) - (height / 4)
        top_left_x = int(top_left_x)
        top_left_y = int(top_left_y)
        bottom_right_x = int(bottom_right_x)
        bottom_right_y = int(bottom_right_y)
        # Display the image with the ROI rectangle
        cv2.rectangle(img, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (0,0,255), 3)
        # Display image
        cv2.imshow("Template", img)
        if cv2.waitKey(1) == 13:
            frame = img
            break
            cv2.destroyAllWindows()
    else:
        break
        cv2.destroyAllWindows()
        
r, h, c, w = 300, 50, 300, 50 
track_window = (c, r, w, h)

# Crop region of interest for tracking  and convert it to HSV color space

roi = frame[r:r+h, c:c+w]
hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

# Create a mask between the HSV bounds
lower_red = np.array([0,0,0])
upper_red = np.array([60,255,255])
mask = cv2.inRange(hsv_roi, lower_red, upper_red)

# Obtain the color histogram of the ROI
roi_hist = cv2.calcHist([hsv_roi], [0], mask, [180], [0,180])

# Normalize values to lie between the range 0, 255
cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True:
    
    # Read webcam frame
    ret, frame = cap.read()

    if ret == True:
        # Convert to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)        
        # Calculate the histogram back projection 
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
        # apply Camshift to get the new location
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        # Draw it on image using polylines to represent the adaptive box 
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame,[pts],True, 255,2)
        
        cv2.imshow('Camshift Tracking', img2)
        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break
    else:
        break
cv2.destroyAllWindows()
cap.release()